In [2]:
import requests
from bs4 import BeautifulSoup

# 웹페이지 URL
url = 'https://data.seoul.go.kr/SeoulRtd/list'

# 페이지의 HTML 가져오기
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# 이미지 URL 찾기
images = soup.find_all('img')
image_urls = [img['src'] for img in images if 'hotspot' in img['src']]

# 이미지 다운로드



In [12]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.webdriver.common.by import By


In [6]:
service = Service(ChromeDriverManager().install())

driver = webdriver.Chrome(service=service)

# 웹페이지 열기
driver.get('https://data.seoul.go.kr/SeoulRtd/list')
time.sleep(5)  # 페이지가 완전히 로드될 시간을 주기

# 페이지의 HTML 가져오기
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')


In [13]:
# 이미지 URL 추출
image_elements = driver.find_elements(By.CSS_SELECTOR, 'img[src*="/SeoulRtd/images/hotspot/"]')
image_urls = [img.get_attribute('currentSrc') for img in image_elements]

In [17]:
page_number = 2

In [18]:
driver.execute_script(f"movePage('{page_number}')")

In [19]:
# 웹 드라이버 설정
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# 웹페이지 열기
driver.get('https://data.seoul.go.kr/SeoulRtd/list')
time.sleep(3)  # 초기 로드 대기

# 페이지 번호 설정
total_pages = 8  # 총 페이지 수, 실제로 확인하여 설정 필요
all_image_urls = []

In [20]:




# 각 페이지마다 이미지 URL 수집
for page_number in range(1, total_pages + 1):
    # 페이지 이동 스크립트 실행
    driver.execute_script(f"movePage('{page_number}')")
    time.sleep(2)  # 페이지 로드 대기

    # 현재 페이지의 이미지 URL 추출
    image_elements = driver.find_elements(By.CSS_SELECTOR, 'img[src*="/SeoulRtd/images/hotspot/"]')
    current_page_image_urls = [img.get_attribute('currentSrc') for img in image_elements]
    all_image_urls.extend(current_page_image_urls)

    # 페이지의 이미지 URL 출력
    print(f"Page {page_number} Image URLs:")
    for url in current_page_image_urls:
        print(url)

# 모든 페이지의 이미지 URL 출력
print("\nAll collected Image URLs:")
for url in all_image_urls:
    print(url)

# 드라이버 종료
driver.quit()


Page 1 Image URLs:
https://data.seoul.go.kr/SeoulRtd/images/hotspot/%EB%B6%81%ED%95%9C%EC%82%B0%EC%9A%B0%EC%9D%B4%EC%97%AD.jpg
https://data.seoul.go.kr/SeoulRtd/images/hotspot/%EB%B6%81%EC%84%9C%EC%9A%B8%EA%BF%88%EC%9D%98%EC%88%B2.jpg
https://data.seoul.go.kr/SeoulRtd/images/hotspot/%EC%95%84%EC%B0%A8%EC%82%B0.jpg
https://data.seoul.go.kr/SeoulRtd/images/hotspot/%EB%B6%88%EA%B4%91%EC%B2%9C.jpg
https://data.seoul.go.kr/SeoulRtd/images/hotspot/%EB%8C%80%EB%A6%BC%EC%97%AD.jpg
https://data.seoul.go.kr/SeoulRtd/images/hotspot/%EC%9D%B4%ED%83%9C%EC%9B%90%20%EC%95%A4%ED%8B%B1%EA%B0%80%EA%B5%AC%EA%B1%B0%EB%A6%AC.jpg
https://data.seoul.go.kr/SeoulRtd/images/hotspot/%EC%B0%BD%EB%8F%99%20%EC%8B%A0%EA%B2%BD%EC%A0%9C%20%EC%A4%91%EC%8B%AC%EC%A7%80.jpg
https://data.seoul.go.kr/SeoulRtd/images/hotspot/%EC%9B%94%EB%93%9C%EC%BB%B5%EA%B3%B5%EC%9B%90.jpg
https://data.seoul.go.kr/SeoulRtd/images/hotspot/%EB%82%99%EC%82%B0%EA%B3%B5%EC%9B%90%C2%B7%EC%9D%B4%ED%99%94%EB%A7%88%EC%9D%84.jpg
https://data.seoul.go

In [29]:
import os

In [31]:
import json

In [32]:
BASE_DIR = os.path.dirname(os.path.dirname(os.path.relpath("./")))
secret_file = os.path.join(BASE_DIR, '../fastapi-server/app/secret.json')


with open(secret_file) as f:
    secrets = json.loads(f.read())

def get_secret(setting, secrets=secrets):
    try:
        return secrets[setting]
    except KeyError:
        errorMsg = "Set the {} environment variable.".format(setting)
        return errorMsg




# MySQL 데이터베이스 연결 설정
username = get_secret('Mysql_Username')
password = get_secret('Mysql_Password')
database = get_secret('Mysql_DBname')
host = get_secret('Mysql_Hostname')
port = get_secret('Mysql_Port')

In [34]:
from sqlalchemy import create_engine

In [35]:
engine = create_engine(f'mysql+mysqldb://{username}:{password}@{host}:{port}/{database}')


In [44]:
# Web driver setup
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Open web page
driver.get('https://data.seoul.go.kr/SeoulRtd/list')
time.sleep(3)  # Allow time for the page to load initially


In [45]:
import urllib.parse


In [46]:



total_pages = 8  # Total number of pages, adjust as necessary
landmark_image_map = {}

# Iterate over each page to collect image URLs
for page_number in range(1, total_pages + 1):
    # Execute page navigation script
    driver.execute_script(f"movePage('{page_number}')")
    time.sleep(5)  # Wait for the page to load

    # Find all anchor tags that contain links to landmarks
    anchor_elements = driver.find_elements(By.CSS_SELECTOR, 'div.srcd_list a[href*="hotspotNm="]')
    
    for anchor in anchor_elements:
        img_element = anchor.find_element(By.TAG_NAME, 'img')
        image_url = img_element.get_attribute('currentSrc')
        time.sleep(1)
        href = anchor.get_attribute('href')
        parsed_url = urllib.parse.urlparse(href)
        query_params = urllib.parse.parse_qs(parsed_url.query)
        landmark_name = query_params.get('hotspotNm', [None])[0]
        
        if landmark_name:
            landmark_image_map[landmark_name] = image_url

    # Print image URLs for the current page
    print(f"Page {page_number} Image URLs:")
    for landmark, url in landmark_image_map.items():
        print(f"{landmark}: {url}")

# Print all collected image URLs
print("\nAll collected Landmark to Image URLs:")
for landmark, url in landmark_image_map.items():
    print(f"{landmark}: {url}")

# Close the driver
driver.quit()


Page 1 Image URLs:
북한산우이역: https://data.seoul.go.kr/SeoulRtd/images/hotspot/%EB%B6%81%ED%95%9C%EC%82%B0%EC%9A%B0%EC%9D%B4%EC%97%AD.jpg
가락시장: https://data.seoul.go.kr/SeoulRtd/images/hotspot/%EA%B0%80%EB%9D%BD%EC%8B%9C%EC%9E%A5.jpg
북서울꿈의숲: https://data.seoul.go.kr/SeoulRtd/images/hotspot/%EB%B6%81%EC%84%9C%EC%9A%B8%EA%BF%88%EC%9D%98%EC%88%B2.jpg
아차산: https://data.seoul.go.kr/SeoulRtd/images/hotspot/%EC%95%84%EC%B0%A8%EC%82%B0.jpg
응봉산: https://data.seoul.go.kr/SeoulRtd/images/hotspot/%EC%9D%91%EB%B4%89%EC%82%B0.jpg
창동 신경제 중심지: https://data.seoul.go.kr/SeoulRtd/images/hotspot/%EC%B0%BD%EB%8F%99%20%EC%8B%A0%EA%B2%BD%EC%A0%9C%20%EC%A4%91%EC%8B%AC%EC%A7%80.jpg
어린이대공원: https://data.seoul.go.kr/SeoulRtd/images/hotspot/%EC%96%B4%EB%A6%B0%EC%9D%B4%EB%8C%80%EA%B3%B5%EC%9B%90.jpg
이태원 앤틱가구거리: https://data.seoul.go.kr/SeoulRtd/images/hotspot/%EC%9D%B4%ED%83%9C%EC%9B%90%20%EC%95%A4%ED%8B%B1%EA%B0%80%EA%B5%AC%EA%B1%B0%EB%A6%AC.jpg
불광천: https://data.seoul.go.kr/SeoulRtd/images/hotspot/%EB%B6%88%EA%B4%9

In [48]:
len(landmark_image_map.keys())

115

In [47]:
for key, value in landmark_image_map.items():
    if value == '':
        print(key, " is empty")

In [50]:
from sqlalchemy import text


In [52]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import SQLAlchemyError


# 세션 생성
Session = sessionmaker(bind=engine)
session = Session()

# 이미지 URL 업데이트 함수
def update_image_url(landmark_name, image_url):
    try:
        # SQL 실행
        session.execute(
            text("UPDATE landmark SET url = :url WHERE 랜드마크 = :landmark"),
            {'url': image_url, 'landmark': landmark_name}
        )
        session.commit()
        print("URL updated successfully")
    except SQLAlchemyError as err:
        print(f"Error: '{err}'")
        session.rollback()



# 각 이미지에 대한 URL을 데이터베이스에 업데이트
for landmark, url in landmark_image_map.items():
    update_image_url(landmark, url)

# 세션 종료
session.close()


URL updated successfully
URL updated successfully
URL updated successfully
URL updated successfully
URL updated successfully
URL updated successfully
URL updated successfully
URL updated successfully
URL updated successfully
URL updated successfully
URL updated successfully
URL updated successfully
URL updated successfully
URL updated successfully
URL updated successfully
URL updated successfully
URL updated successfully
URL updated successfully
URL updated successfully
URL updated successfully
URL updated successfully
URL updated successfully
URL updated successfully
URL updated successfully
URL updated successfully
URL updated successfully
URL updated successfully
URL updated successfully
URL updated successfully
URL updated successfully
URL updated successfully
URL updated successfully
URL updated successfully
URL updated successfully
URL updated successfully
URL updated successfully
URL updated successfully
URL updated successfully
URL updated successfully
URL updated successfully
